# **Chapter 1: Basic Prompt Structure**

---
**Lesson:**

When using the console or API, the most effective prompts for Mistral AI models should be formatted in a specific manner by marking the beginning of the user input with `<s>`[INST] and end it with [/INST]. Everything that is not present in between these tags are the generated responses from the model. This notebook focuses on Mistral 7B, but all concepts apply to Mixtral 8x7B as well. For more information on prompting Mistral AI models, please refer to this post [How to Prompt Mistral AI models, and Why](https://community.aws/content/2dFNOnLVQRhyrOrMsloofnW0ckZ/how-to-prompt-mistral-ai-models-and-why)

---

First we will setup our dependencies

In [ ]:
%%capture
#Install dependencies
%pip install --no-build-isolation --force-reinstall \
    "boto3>=1.28.57" \
    "awscli>=1.29.57" \
    "botocore>=1.31.57"

#Import libraries, and set up Bedrock client
import json
import os
import sys

import boto3
import botocore.exceptions

module_path = ".."
sys.path.append(os.path.abspath(module_path))
from utils import bedrock, print_ww

boto3_bedrock = bedrock.get_bedrock_client(
    assumed_role=os.environ.get("BEDROCK_ASSUME_ROLE", None),
    region=os.environ.get("AWS_DEFAULT_REGION", None)
)

modelId = "mistral.mistral-7b-instruct-v0:2" # change this to use a different version from the model provider
accept = 'application/json'
contentType = 'application/json'
outputText = "\n"

def invoke_model_and_get_response(prompt_data): 
    body = json.dumps({ 
        'prompt': prompt_data,
        'max_tokens': 400,
        'top_p': 0.7,
        'top_k': 50,
        'temperature': 0.7
    })

    try:
        response = boto3_bedrock.invoke_model(body=body, modelId=modelId, accept=accept, contentType=contentType)
        response_body = json.loads(response.get('body').read().decode('utf-8'))
        outputs = response_body["outputs"]
        outputText = [output["text"] for output in outputs]
        return outputText

    except botocore.exceptions.ClientError as error:
        if error.response['Error']['Code'] == 'AccessDeniedException':
            return (f"\x1b[41m{error.response['Error']['Message']}\
                    \nTo troubleshoot this issue please refer to the following resources.\
                     \nhttps://docs.aws.amazon.com/IAM/latest/UserGuide/troubleshoot_access-denied.html\
                     \nhttps://docs.aws.amazon.com/bedrock/latest/userguide/security-iam.html\x1b[0m\n")
        else:
            raise

# **Examples:**

Let's take a look at how Mistral responds to some *correctly-formatted prompts*. Don't worry about how the answers are being generated for now. Please run the example cells below to get the response from Mistral 7B.

**Example 1.1 - How are you?**

In [ ]:
# create the prompt
prompt_data = """<s>[INST]Hi, how are you?[\INST]"""

response = invoke_model_and_get_response(prompt_data)
print(response)

**Example 1.2 - The Sky**

In [ ]:
# create the prompt
prompt_data = """<s>[INST]Can you tell me the color of the sky?[\INST]"""

response = invoke_model_and_get_response(prompt_data)
print(response)

# **Exercises**

The following two exercises will need you to manipulate the prompt to get the desired output

**Exercise 1.1 - Happy Birthday**

Using proper formatting, fix the prompt in the cell below to get Mistral to say a happy/funny birthday message filled with jokes.

In [ ]:
# create the prompt
prompt_data = """<s>[INST]Send a boring happy birthday message to Bob[\INST]"""

response = invoke_model_and_get_response(prompt_data)
print(response)

**Exercise 1.2 - Favourite Colors**

What's wrong with the following prompt? If you fix it, Mistral 7B will tell you some colors it likes. Fix the following prompt so Mistral 7B's response cell mentions some colours. 

> *Note: Running the cell below will give you a poor and confusing answer - how can you fix the prompt?*

In [ ]:
# create the prompt
prompt_data = """<s>[INST]Can you tell me your favourite colour?[\INST]

Well.... I don't exactly have a favourite color because I can't see ;) But there are some colors I like.

"""

response = invoke_model_and_get_response(prompt_data)
print(response)

# Chapter 1 - END.